In [40]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
import os

In [41]:
# Define paths (Train and Test main folders)
train_dir = r"C:\Users\rpaka\OneDrive\Desktop\dataset\train"
test_dir  = r"C:\Users\rpaka\OneDrive\Desktop\dataset\test"

In [42]:
# Load training dataset
train_ds = image_dataset_from_directory(
    train_dir,
    image_size=(224, 224),
    batch_size=32,
    label_mode="categorical"   # multi-class classification
)

Found 125 files belonging to 2 classes.


In [43]:
# Load testing dataset
test_ds = image_dataset_from_directory(
    test_dir,
    image_size=(224, 224),
    batch_size=32,
    label_mode="categorical"
)

Found 40 files belonging to 2 classes.


In [44]:
class_names = train_ds.class_names
print("Classes found:",class_names)


Classes found: ['non_violent', 'violent']


In [45]:
train_ds = train_ds.map(lambda x, y:(x/255.0,y))
test_ds = test_ds.map(lambda x, y:(x/255.0,y))


In [46]:
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(len(class_names), activation='softmax')  # 4 classes
])

In [47]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [48]:
history = model.fit(train_ds, validation_data=test_ds,epochs=10)

Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.6480 - loss: 0.6596 - val_accuracy: 0.7000 - val_loss: 0.3312
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.8560 - loss: 0.3038 - val_accuracy: 1.0000 - val_loss: 0.0473
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 1.0000 - loss: 0.0280 - val_accuracy: 1.0000 - val_loss: 0.0016
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.9920 - loss: 0.0088 - val_accuracy: 1.0000 - val_loss: 1.8220e-04
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.9920 - loss: 0.0157 - val_accuracy: 1.0000 - val_loss: 0.0013
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.9920 - loss: 0.0100 - val_accuracy: 1.0000 - val_loss: 2.1746e-05
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 1.0000 - loss: 3.8615e-04 - val_accuracy: 1.0000 - val_loss: 0.0023
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.9920 - loss: 0.0098 - val_accuracy: 1.0000 - val_loss: 2.1621e-05

In [50]:
y_pred = []
y_true = []

# Iterate over the test dataset
for images, labels in test_ds:
    preds = model(images, training=False)  # safer than model.predict
    preds = tf.argmax(preds, axis=1).numpy()      # convert to numpy array
    true_labels = tf.argmax(labels, axis=1).numpy()
    
    y_pred.extend(preds)
    y_true.extend(true_labels)

y_pred = np.array(y_pred)
y_true = np.array(y_true)

In [52]:
cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:\n", cm)

Confusion Matrix:
 [[28  0]
 [ 0 12]]
